In [1]:

import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

path = '/kaggle/input/clean-dirty-containers-in-montevideo/clean-dirty-garbage-containers-V4/clean-dirty-garbage-containers'
#eval_path = '/kaggle/input/clean-dirty-containers-in-montevideo/clean-dirty-garbage-containers-V4/clean-dirty-garbage-containers/test'
#weigths='/kaggle/input/keras-pretrain-model-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import wandb

In [4]:
class ImageDataset(Dataset):
    
    def __init__(self,path):
        super().__init__()
        self.path = path
        self.train = os.listdir(path)
        self.image_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),])
   
    def __len__(self):
        return len(self.train)
    
    def __getitem__(self,idx):
        image = f'{self.path}/{self.train[idx]}'
        image = Image.open(image)
        if self.image_transforms:
            return self.image_transforms(image)
        return image
    

def make_dataloader(batch_size=8, **kwargs):
    dataset = ImageDataset(**kwargs)
    dataloader = DataLoader(dataset,batch_size=batch_size,num_workers=1,pin_memory=True,shuffle=True)
    return dataloader  

In [5]:
dataloader_traindirty = make_dataloader(path=f'{path}/train/dirty')
dataloader_trainclean = make_dataloader(path=f'{path}/train/clean')
dataloader_testclean = make_dataloader(path=f'{path}/test/clean')
dataloader_testdirty = make_dataloader(path=f'{path}/test/dirty')

In [6]:
def imshow(grid,title):
    nparray = grid.cpu().numpy()
    nparray = (nparray*0.5)+0.5
    plt.figure(figsize=(12,12))
    plt.title(title)
    plt.axis('off')
    plt.imshow(np.transpose(nparray,(1,2,0)))
    plt.show()

In [7]:
imgsA = next(iter(dataloader_trainclean))
gridA = torchvision.utils.make_grid(imgsA,nrow=4)
imshow(gridA,'Clean Image')

In [8]:
imgsA = next(iter(dataloader_traindirty))
gridA = torchvision.utils.make_grid(imgsA,nrow=4)
imshow(gridA,'Dirty Image')

In [24]:
class ConvBlock(nn.Module):
  
    def __init__(self,in_channels,out_channels,down=True,act=True,**kwargs):
        super().__init__()
        self.conv = nn.Sequential(
                             nn.Conv2d(in_channels=in_channels,out_channels=out_channels,padding_mode='reflect',**kwargs)
                             if down else nn.ConvTranspose2d(in_channels=in_channels,out_channels=out_channels,**kwargs),
                             nn.InstanceNorm2d(out_channels),
                             nn.ReLU(inplace=True) if act else nn.Identity(),
                                )

    def forward(self,x):
        return self.conv(x)
    

class ResidualBlock(nn.Module):
  
    def __init__(self,channels,**kwargs):
        super().__init__()
        self.main = nn.Sequential(
                             ConvBlock(channels,channels,kernel_size=3, padding=1),
                             ConvBlock(channels,channels,act=False,kernel_size=3, padding=1)
                                 )

    def forward(self,x):
        return x + self.main(x)
    

class Generator(nn.Module):

    def __init__(self,nb_features,nb_residualblock=9):
        super().__init__()
        self.first = ConvBlock(3,nb_features,kernel_size=7,stride=1,padding=3)
        self.down = nn.ModuleList(
                        [ConvBlock(nb_features,nb_features*2,kernel_size=3, stride=2, padding=1),
                        ConvBlock(nb_features*2,nb_features*4,kernel_size=3, stride=2, padding=1)]
                                 )
        self.residual_block = nn.Sequential(*[ResidualBlock(nb_features*4) for _ in range(nb_residualblock)])
        self.up = nn.ModuleList(
                        [ConvBlock(nb_features*4,nb_features*2,down=False,kernel_size=3, stride=2, padding=1, output_padding=1),
                        ConvBlock(nb_features*2,nb_features,down=False,kernel_size=3, stride=2, padding=1, output_padding=1)]
                               )
        self.last = nn.Conv2d(nb_features,3,kernel_size=7,stride=1,padding=3, padding_mode="reflect")

    def forward(self,x):
        x = self.first(x)
        for layer in self.down:
            x = layer(x)
        x = self.residual_block(x)
        for layer in self.up:
            x = layer(x)
        x = nn.Tanh()(self.last(x))
        return x

In [25]:
class Block(nn.Module):

    def __init__(self,in_channels,out_channels,stride):
        super().__init__()
        self.conv = nn.Sequential(
                          nn.Conv2d(in_channels,out_channels,kernel_size=4,stride=stride,padding=1,padding_mode='reflect'),
                          nn.InstanceNorm2d(out_channels),
                          nn.LeakyReLU(0.2,inplace=True),
                             )
    def forward(self,x):
        return self.conv(x)
    

class Discriminator(nn.Module):

    def __init__(self,nb_features):
        super().__init__()
        self.first = nn.Sequential(
                         nn.Conv2d(3,nb_features,kernel_size=4,stride=2,padding=1,padding_mode='reflect'),
                         nn.LeakyReLU(0.2,inplace=True),
                              )
        self.convblock = nn.ModuleList(
                              [Block(nb_features,nb_features*2,2),
                               Block(nb_features*2,nb_features*4,2),
                               Block(nb_features*4,nb_features*6,1)]
                                  )
        self.last = nn.Sequential(
                        nn.Conv2d(nb_features*6,1,kernel_size=4,padding=1,padding_mode='reflect'),
                        nn.Sigmoid()
                             )
    def forward(self,x):
        x = self.first(x)
        for layer in self.convblock:    
            x = layer(x)
        x = self.last(x)
        return x    

In [26]:
class CycleGAN(nn.Module):
    
    def __init__(self,nb_features=64,lr=0.002,beta1=0.5,beta2=0.999,lambda_cycle=10.0):
        super().__init__()
        self.G_AB = torch.load("../input/cyclegan/CycleGAN/Generator_AtoB.pth").to(device)
        self.G_BA = torch.load("../input/cyclegan/CycleGAN/Generator_BtoA.pth").to(device)
        self.D_A  = torch.load("../input/cyclegan/CycleGAN/Discriminator_A.pth").to(device)
        self.D_B  = torch.load("../input/cyclegan/CycleGAN/Discriminator_B.pth").to(device)
        self.adversarial_loss = nn.MSELoss()
        self.cycle_loss = nn.L1Loss()
        self.opt_G = torch.optim.Adam(itertools.chain(self.G_AB.parameters(),self.G_BA.parameters()),lr=lr,betas=(beta1,beta2))
        self.opt_D_A  = torch.optim.Adam(self.D_A.parameters(),lr=lr,betas=(beta1,beta2))
        self.opt_D_B  = torch.optim.Adam(self.D_B.parameters(),lr=lr,betas=(beta1,beta2))
        self.lambda_cycle = lambda_cycle
    
    def setup_input(self,real_A,real_B):
        self.real_A = real_A.to(device)
        self.real_B = real_B.to(device)
        self.fake_A = self.G_BA(self.real_B)
        self.fake_B = self.G_AB(self.real_A)
    
    def optimize_D(self):
        self.D_A.train()
        self.D_B.train()
        real_preds = self.D_A(self.real_A)
        fake_preds = self.D_A(self.fake_A.detach())
        real_loss = self.adversarial_loss(real_preds,torch.ones_like(real_preds,device=device))
        fake_loss = self.adversarial_loss(fake_preds,torch.zeros_like(fake_preds,device=device))
        loss_D_A = (real_loss+fake_loss)/2
        
        real_preds = self.D_B(self.real_B)
        fake_preds = self.D_B(self.fake_B.detach())
        real_loss = self.adversarial_loss(real_preds,torch.ones_like(real_preds,device=device))
        fake_loss = self.adversarial_loss(fake_preds,torch.zeros_like(fake_preds,device=device))
        loss_D_B = (real_loss+fake_loss)/2
    
        self.opt_D_A.zero_grad()
        loss_D_A.backward()
        self.opt_D_A.step()
        
        self.opt_D_B.zero_grad()
        loss_D_B.backward()
        self.opt_D_B.step()
        
        return loss_D_A,loss_D_B
        
    def optimize_G(self):
        self.G_AB.train()
        self.G_BA.train()
        fake_preds_A = self.D_A(self.fake_A)
        fake_preds_B = self.D_B(self.fake_B)
        adversarial_loss_G_AB = self.adversarial_loss(fake_preds_B,torch.ones_like(fake_preds_B,device=device))
        adversarial_loss_G_BA = self.adversarial_loss(fake_preds_A,torch.ones_like(fake_preds_A,device=device))
        adversarial_loss_G = (adversarial_loss_G_AB + adversarial_loss_G_BA)/2
        
        cycle_loss_G_AB = self.cycle_loss(self.real_A,self.G_BA(self.fake_B))
        cycle_loss_G_BA = self.cycle_loss(self.real_B,self.G_AB(self.fake_A))
        cycle_loss_G = (cycle_loss_G_AB + cycle_loss_G_BA)/2
        loss_G = adversarial_loss_G + (self.lambda_cycle*cycle_loss_G)
    
        self.opt_G.zero_grad()
        loss_G.backward()
        self.opt_G.step()
        
        return loss_G

In [27]:
gan = CycleGAN()

In [9]:
import torch.nn as nn
import torch.nn.functional as F

# helper conv function
def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                           kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    
    layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)

In [10]:
class Discriminator(nn.Module):
    
    def __init__(self, conv_dim=64):
        super(Discriminator, self).__init__()

        # Define all convolutional layers
        # Should accept an RGB image as input and output a single value
        
        # 1st Conv layer has no batch_norm. dim = [64, 64, 64]
        self.conv1 = conv(3, conv_dim, 4, batch_norm=False)
        # 2nd Conv layer. dim = [32, 32, 128]
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        # 3rd Conv layer. dim = [16, 16, 256]
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        # 4th Conv layer. dim = [8, 8, 512]
        self.conv4 = conv(conv_dim*4, conv_dim*8, 4)
        # 5th Last Conv layer, Classification Layer has no batch_norm and stride is 1. dim = [1, 1, 1]
        self.conv5 = conv(conv_dim*8, 1, 4, stride=1, batch_norm=False)

    def forward(self, x):
        # feedforward behavior - relu to all but last layer
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.conv5(x)

        return x

In [11]:
# residual block class
class ResidualBlock(nn.Module):
    """Defines a residual block.
       This adds an input x to a convolutional layer (applied to x) with the same size input and output.
       These blocks allow a model to learn an effective transformation from one domain to another.
    """
    def __init__(self, conv_dim):
        super(ResidualBlock, self).__init__()
        # conv_dim = number of inputs  
        
        # define two convolutional layers + batch normalization that will act as our residual function, F(x)
        # layers should have the same shape input as output; I suggest a kernel_size of 3
        self.conv1 = conv(conv_dim, conv_dim, 3, 1, batch_norm=True)
        self.conv2 = conv(conv_dim, conv_dim, 3, 1, batch_norm=True)
        
    def forward(self, x):
        # apply a ReLu activation the outputs of the first layer
        # return a summed output, x + resnet_block(x)
        output = F.relu(self.conv1(x))
        output = x + self.conv2(output)
        
        return output
    

In [12]:
class CycleGenerator(nn.Module):
    
    def __init__(self, conv_dim=64, n_res_blocks=6):
        super(CycleGenerator, self).__init__()

        # 1. Define the encoder part of the generator
        self.conv1 = conv(3, conv_dim, 4)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)

        # 2. Define the resnet part of the generator
        res_layers = []
        for layer in range(n_res_blocks):
          res_layers.append(ResidualBlock(conv_dim*4))

        self.resblocks = nn.Sequential(*res_layers)

        # 3. Define the decoder part of the generator
        self.deconv1 = deconv(conv_dim*4, conv_dim*2, 4)
        self.deconv2 = deconv(conv_dim*2, conv_dim, 4)
        self.deconv3 = deconv(conv_dim, 3, 4, batch_norm=False)

    def forward(self, x):
        """Given an image x, returns a transformed image."""
        # define feedforward behavior, applying activations as necessary
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        x = self.resblocks.forward(x)

        x = F.relu(self.deconv1(x))
        x = F.relu(self.deconv2(x))
        x = F.tanh(self.deconv3(x))

        return x

In [13]:
def create_model(g_conv_dim=64, d_conv_dim=64, n_res_blocks=6):
    """Builds the generators and discriminators."""
    
    # Instantiate generators
    G_XtoY = CycleGenerator(conv_dim=g_conv_dim, n_res_blocks=n_res_blocks)
    G_YtoX = CycleGenerator(conv_dim=g_conv_dim, n_res_blocks=n_res_blocks)
    # Instantiate discriminators
    D_X = Discriminator(conv_dim=d_conv_dim)
    D_Y = Discriminator(conv_dim=d_conv_dim)

    # move models to GPU, if available
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        G_XtoY.to(device)
        G_YtoX.to(device)
        D_X.to(device)
        D_Y.to(device)
        print('Models moved to GPU.')
    else:
        print('Only CPU available.')

    return G_XtoY, G_YtoX, D_X, D_Y

In [14]:
# helper deconv function
def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    """Creates a transpose convolutional layer, with optional batch normalization.
    """
    layers = []
    # append transpose conv layer
    layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False))
    # optional batch norm layer
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)

In [15]:
# call the function to get models
G_XtoY, G_YtoX, D_X, D_Y = create_model()

In [16]:
# helper function for printing the model architecture
def print_models(G_XtoY, G_YtoX, D_X, D_Y):
    """Prints model information for the generators and discriminators.
    """
    print("                     G_XtoY                    ")
    print("-----------------------------------------------")
    print(G_XtoY)
    print()

    print("                     G_YtoX                    ")
    print("-----------------------------------------------")
    print(G_YtoX)
    print()

    print("                      D_X                      ")
    print("-----------------------------------------------")
    print(D_X)
    print()

    print("                      D_Y                      ")
    print("-----------------------------------------------")
    print(D_Y)
    print()
    

# print all of the models
print_models(G_XtoY, G_YtoX, D_X, D_Y)

In [17]:
def real_mse_loss(D_out):
    # how close is the produced output from being "real"?
    return torch.mean((D_out-1)**2)

def fake_mse_loss(D_out):
    # how close is the produced output from being "fake"?
    return torch.mean((D_out-0)**2)

def cycle_consistency_loss(real_im, reconstructed_im, lambda_weight):
    # calculate reconstruction loss 
    reconstr_loss = torch.mean(torch.abs(real_im - reconstructed_im))
    # return weighted loss
    return lambda_weight*reconstr_loss

In [18]:
import torch.optim as optim

# hyperparams for Adam optimizers
lr=0.0002
beta1=0.5
beta2=0.999

g_params = list(G_XtoY.parameters()) + list(G_YtoX.parameters())  # Get generator parameters

# Create optimizers for the generators and discriminators
g_optimizer = optim.Adam(g_params, lr, [beta1, beta2])
d_x_optimizer = optim.Adam(D_X.parameters(), lr, [beta1, beta2])
d_y_optimizer = optim.Adam(D_Y.parameters(), lr, [beta1, beta2])

In [22]:
# train the network
def training_loop(dataloader_X, dataloader_Y, test_dataloader_X, test_dataloader_Y, 
                  n_epochs=1000):
    
    print_every=10
    
    # keep track of losses over time
    losses = []

    test_iter_X = iter(test_dataloader_X)
    test_iter_Y = iter(test_dataloader_Y)

    # Get some fixed data from domains X and Y for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    #fixed_X = test_iter_X.next()[0]
    #fixed_Y = test_iter_Y.next()[0]
    #fixed_X = scale(fixed_X) # make sure to scale to a range -1 to 1
    #fixed_Y = scale(fixed_Y)

    # batches per epoch
    iter_X = iter(dataloader_X)
    iter_Y = iter(dataloader_Y)
    batches_per_epoch = min(len(iter_X), len(iter_Y))

    for epoch in range(1, n_epochs+1):

        # Reset iterators for each epoch
        if epoch % batches_per_epoch == 0:
            iter_X = iter(dataloader_X)
            iter_Y = iter(dataloader_Y)

        images_X, _ = iter_X.next()
        #images_X = scale(images_X) # make sure to scale to a range -1 to 1

        images_Y, _ = iter_Y.next()
        #images_Y = scale(images_Y)
        
        # move images to GPU if available (otherwise stay on CPU)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        images_X = images_X.to(device)
        images_Y = images_Y.to(device)


        # ============================================
        #            TRAIN THE DISCRIMINATORS
        # ============================================

        ##   First: D_X, real and fake loss components   ##
        d_x_optimizer.zero_grad()
        # 1. Compute the discriminator losses on real images
        out_x = D_X(images_X)
        D_X_real_loss = real_mse_loss(out_x)

        # 2. Generate fake images that look like domain X based on real images in domain Y
        fake_x = G_YtoX(images_Y)

        # 3. Compute the fake loss for D_X
        out_x = D_X(fake_x)
        D_X_fake_loss = fake_mse_loss(out_x)

        # 4. Compute the total loss and perform backprop
        d_x_loss = D_X_real_loss + D_X_fake_loss
        d_x_loss.backward(retain_graph=True)
        d_x_optimizer.step()
        
        ##   Second: D_Y, real and fake loss components   ##
        d_y_optimizer.zero_grad()

        # 1. Compute the discriminator losses on real images
        out_y = D_Y(images_Y)
        D_Y_real_loss = real_mse_loss(out_y)

        # 2. Generate fake images that look like domain Y based on real images in domain X
        fake_y = G_XtoY(images_X)

        # 3. Compute the fake loss for D_Y
        out_y = D_Y(fake_y)
        D_Y_fake_loss = fake_mse_loss(out_y)

        # 4. Compute the total loss and perform backprop
        d_y_loss = D_Y_real_loss + D_Y_fake_loss
        d_y_loss.backward(retain_graph=True)
        d_y_optimizer.step()
        

        # =========================================
        #            TRAIN THE GENERATORS
        # =========================================

        ##    First: generate fake X images and reconstructed Y images    ##
        g_optimizer.zero_grad()
        
        # 1. Generate fake images that look like domain X based on real images in domain Y
        fake_X = G_YtoX(images_Y)

        # 2. Compute the generator loss based on domain X
        out_x = D_X(fake_X)
        g_YtoX_loss = real_mse_loss(out_x)

        # 3. Create a reconstructed y
        reconstructed_Y = G_XtoY(fake_X)

        # 4. Compute the cycle consistency loss (the reconstruction loss)
        reconstructed_y_loss = cycle_consistency_loss(images_Y, reconstructed_Y, lambda_weight=10)

        ##    Second: generate fake Y images and reconstructed X images    ##
         
        # 1. Generate fake images that look like domain X based on real images in domain Y
        fake_Y = G_XtoY(images_X)

        # 2. Compute the generator loss based on domain X
        out_x = D_Y(fake_Y)
        g_XtoY_loss = real_mse_loss(out_y)

        # 3. Create a reconstructed y
        reconstructed_X = G_YtoX(fake_Y)

        # 4. Compute the cycle consistency loss (the reconstruction loss)
        reconstructed_x_loss = cycle_consistency_loss(images_X, reconstructed_X, lambda_weight=10)


        # 5. Add up all generator and reconstructed losses and perform backprop
        g_total_loss = g_YtoX_loss + g_XtoY_loss + reconstructed_y_loss + reconstructed_x_loss
      
        g_total_loss.backward(retain_graph=True)
        g_optimizer.step()
        
        # Print the log info
        if epoch % print_every == 0:
            # append real and fake discriminator losses and the generator loss
            losses.append((d_x_loss.item(), d_y_loss.item(), g_total_loss.item()))
            print('Epoch [{:5d}/{:5d}] | d_X_loss: {:6.4f} | d_Y_loss: {:6.4f} | g_total_loss: {:6.4f}'.format(
                    epoch, n_epochs, d_x_loss.item(), d_y_loss.item(), g_total_loss.item()))

            
        sample_every=100
        # Save the generated samples
        if epoch % sample_every == 0:
            G_YtoX.eval() # set generators to eval mode for sample generation
            G_XtoY.eval()
            save_samples(epoch, G_YtoX, G_XtoY, batch_size=16)
            G_YtoX.train()
            G_XtoY.train()

        # uncomment these lines, if you want to save your model
#         checkpoint_every=1000
#         # Save the model parameters
#         if epoch % checkpoint_every == 0:
#             checkpoint(epoch, G_XtoY, G_YtoX, D_X, D_Y)

    return losses


In [23]:
n_epochs = 10 # keep this small when testing if a model first works, then increase it to >=1000
losses = training_loop(dataloader_traindirty, dataloader_trainclean, dataloader_testdirty, dataloader_testclean, n_epochs=n_epochs)